In [24]:
import librosa
import soundfile
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [25]:
# Function for extracting mfcc, chroma and mel features from sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
            # print("Output of mfcc: \n", result)
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
            # print("Output of chroma: \n", result)
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
            # print("Output of mel: \n", result)
    return result

In [26]:
# Emotions in RAVDESS Dataset
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Emotions we want to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

In [27]:
# Load the data and extract features from each sound file
# Load the data and extract features for each sound file
def load_data(test_size=0.20):
    x, y = [], []
    for file in glob.glob("D:\Speech Emotion Recognition (Project)\ser-ravdess-data\Actor_*\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [28]:
# Splitting the dataset into training and test sets 
# (with test size of 25% of the whole dataset)
x_train, x_test, y_train, y_test = load_data(0.25)

In [29]:
# Shape of train and test set and number of features extracted
print((x_train.shape[0], x_test.shape[0]))
print("Features extracted: {}".format(x_train.shape[1]))

(576, 192)
Features extracted: 180


In [30]:
# Initialse Multi Layer Perceptron Classifier
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, 
hidden_layer_sizes=(300, ), learning_rate='adaptive', max_iter=500)

In [31]:
# Train the model
model.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [32]:
# Predicting for the test set
y_pred = model.predict(x_test)

In [33]:
# Calculating accuracy for our model
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 56.77%
